# Introduction

Last chapter we see how DQN works. This chapter we take a look at hwat could be done to improve DQN. We will introduce Double-DQN and Duel-DQN, with slight modification based on DQN and significant improvement on performance.

# Double DQN
The standard DQN is notoriously famous for its possible overestimation on $Q$. Why is so? The TD estimation of DQN is $r + \gamma \max_{a'} Q_{\omega^-}(s', a')$. We can also write $\max_{a'} Q_{\omega^-}(s', a')$ as $$Q_{\omega^-}(s', \max_{a'} Q_{\omega^-}(s', a')).$$ In other words, the $\max$ operation here can be divided into 2 steps: 1. select the best action $s'$ under state $s$, and 2. calculate the corresponding value $Q_\omega(s', a')$. Using the same Q-netowrk for the two calculation, each time we will get the maximum under current estimation. Since the network may have positive or negative bias, it is easy to see that the positive bias will be accumulated. For a large action space, such deficiency may break DQN totally. 

Motivative by this idea, Double-DQN solves the overestimation issue by use 2 networks for esitmation: $$Q_{\omega^-}(s', \max_{a'} Q_{\omega}(s', a')),$$ where $Q_{\omega^-}$ and $Q_{\omega}$ are 2 different networks. By doing so, although $Q_{\omega}$ may still overstimate every time, $Q_{\omega^-}$ could have either positive or negative bias, which alleviates the overestimation. 

In traditional DQN, since we already have 2 Q-networks, we can directly use the training network for action selection, and the target network for value estimation. We can write out the optimization object directly:  $$ r + \gamma  * Q_{\omega^-}(s', \max_{a'} Q_{\omega}(s', a'))$$